In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
#import os
#os.environ['OMP_NUM_THREADS'] = '4'
import gc

path = '/home/kai/data/kaggle/talkingdata/data/'
dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }
print('load train....')
# we save only day 9

load train....


In [3]:
print('load train....')
# we save only day 9 (changed 131886954 to 171886954)
train_df = pd.read_csv(path+"train.csv", dtype=dtypes, skiprows = range(1, 171886954), usecols=['ip','app','device','os', 'channel', 'click_time', 'is_attributed'])
print('load test....')
test_df = pd.read_csv(path+"test.csv", dtype=dtypes, usecols=['ip','app','device','os', 'channel', 'click_time', 'click_id'])

load train....
load test....


In [4]:
len_train = len(train_df)
train_df=train_df.append(test_df)
del test_df; gc.collect()

14

In [5]:
print('hour, day, wday....')
train_df['hour'] = pd.to_datetime(train_df.click_time).dt.hour.astype('uint8')
train_df['day'] = pd.to_datetime(train_df.click_time).dt.day.astype('uint8')
train_df['wday']  = pd.to_datetime(train_df.click_time).dt.dayofweek.astype('uint8')

hour, day, wday....


In [6]:
train_df.shape

(31807406, 11)

In [7]:
train_df.head(1)

,app,channel,click_id,click_time,device,ip,is_attributed,os,hour,day,wday
0,9,127,NaN,2017-11-09 12:04:44,1,320071,0.0,19,12,9,3


In [8]:
train_s = train_df.sample(frac=0.000001)

In [9]:
train_s.shape

(32, 11)

In [10]:
train_s[['ip','day','hour','channel']].groupby(by=['ip','day','hour'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'qty'}).head(2)

,ip,day,hour,qty
0,6227,9,12,1
1,6462,9,12,1


In [11]:
print('grouping by ip-day-hour combination....')
gp = train_df[['ip','day','hour','channel']].groupby(by=['ip','day','hour'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'qty'})
train_df = train_df.merge(gp, on=['ip','day','hour'], how='left')
del gp; gc.collect()

grouping by ip-day-hour combination....


106

In [12]:
print('group by ip-app combination....')
gp = train_df[['ip','app', 'channel']].groupby(by=['ip', 'app'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'ip_app_count'})
train_df = train_df.merge(gp, on=['ip','app'], how='left')
del gp; gc.collect()

group by ip-app combination....


101

In [13]:
print('group by ip-app-os combination....')
gp = train_df[['ip','app', 'os', 'channel']].groupby(by=['ip', 'app', 'os'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'ip_app_os_count'})
train_df = train_df.merge(gp, on=['ip','app', 'os'], how='left')
del gp; gc.collect()

group by ip-app-os combination....


117

In [14]:
print("vars and data type....")
train_df['qty'] = train_df['qty'].astype('uint16')
train_df['ip_app_count'] = train_df['ip_app_count'].astype('uint16')
train_df['ip_app_os_count'] = train_df['ip_app_os_count'].astype('uint16')

vars and data type....


In [15]:
print("label encoding....")
from sklearn.preprocessing import LabelEncoder
train_df[['app','device','os', 'channel', 'hour', 'day', 'wday']] = train_df[['app','device','os', 'channel', 'hour', 'day', 'wday']].apply(LabelEncoder().fit_transform)

label encoding....


In [16]:
train_df.tail()

,app,channel,click_id,click_time,device,ip,is_attributed,os,hour,day,wday,qty,ip_app_count,ip_app_os_count
31807401,9,33,18790464.0,2017-11-10 15:00:00,1,99442,NaN,13,9,1,1,2,16,3
31807402,23,45,18790465.0,2017-11-10 15:00:00,1,88046,NaN,37,9,1,1,1,48,2
31807403,18,85,18790467.0,2017-11-10 15:00:00,1,81398,NaN,17,9,1,1,2,243,9
31807404,27,28,18790466.0,2017-11-10 15:00:00,1,123236,NaN,13,9,1,1,1,2,1
31807405,12,85,18790468.0,2017-11-10 15:00:00,2,73516,NaN,27,9,1,1,4,33563,234


In [17]:
print ('final part of preparation....')
test_df = train_df[len_train:]
train_df = train_df[:len_train]
y_train = train_df['is_attributed'].values
train_df.drop(['click_id', 'click_time','ip','is_attributed'],1,inplace=True)

final part of preparation....


In [18]:
train_df.head(2)

,app,channel,device,os,hour,day,wday,qty,ip_app_count,ip_app_os_count
0,9,33,1,19,6,0,0,59,13,4
1,2,64,1,19,6,0,0,47,45,5


In [19]:
test_df.head(2)

,app,channel,click_id,click_time,device,ip,is_attributed,os,hour,day,wday,qty,ip_app_count,ip_app_os_count
13016937,9,17,0.0,2017-11-10 04:00:00,1,5744,NaN,3,0,1,1,34,47,2
13016938,9,164,1.0,2017-11-10 04:00:00,1,119901,NaN,3,0,1,1,403,466,7


In [20]:
len_train

13016937

In [1]:
print ('neural network....')
from keras.layers import Input, Embedding, Dense, Flatten, Dropout, concatenate
from keras.layers import BatchNormalization, SpatialDropout1D
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam

neural network....


/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [22]:
max_app = np.max([train_df['app'].max(), test_df['app'].max()])+1
max_ch = np.max([train_df['channel'].max(), test_df['channel'].max()])+1
max_dev = np.max([train_df['device'].max(), test_df['device'].max()])+1
max_os = np.max([train_df['os'].max(), test_df['os'].max()])+1
max_h = np.max([train_df['hour'].max(), test_df['hour'].max()])+1
max_d = np.max([train_df['day'].max(), test_df['day'].max()])+1
max_wd = np.max([train_df['wday'].max(), test_df['wday'].max()])+1
max_qty = np.max([train_df['qty'].max(), test_df['qty'].max()])+1
max_c1 = np.max([train_df['ip_app_count'].max(), test_df['ip_app_count'].max()])+1
max_c2 = np.max([train_df['ip_app_os_count'].max(), test_df['ip_app_os_count'].max()])+1

In [23]:
max_app, max_c1, max_c2, max_ch, max_d, max_dev, max_h, max_os, max_qty, max_wd

(474, 54117, 11037, 184, 2, 2339, 11, 461, 43959, 2)

In [24]:
def get_keras_data(dataset):
    X = {
        'app': np.array(dataset.app),
        'ch': np.array(dataset.channel),
        'dev': np.array(dataset.device),
        'os': np.array(dataset.os),
        'h': np.array(dataset.hour),
        'd': np.array(dataset.day),
        'wd': np.array(dataset.wday),
        'qty': np.array(dataset.qty),
        'c1': np.array(dataset.ip_app_count),
        'c2': np.array(dataset.ip_app_os_count)
    }
    return X

In [25]:
train_df = get_keras_data(train_df)

In [26]:
len(train_df['app']), len(train_df['c2'])

(13016937, 13016937)

In [27]:
emb_n = 50
dense_n = 1000

In [28]:
in_app = Input(shape=[1], name = 'app')
emb_app = Embedding(max_app, emb_n)(in_app)

In [29]:
in_ch = Input(shape=[1], name = 'ch1')
emb_ch = Embedding(max_ch, emb_n)(in_ch)
in_dev = Input(shape=[1], name = 'dev')
emb_dev = Embedding(max_dev, emb_n)(in_dev)
in_os = Input(shape=[1], name = 'os')
emb_os = Embedding(max_os, emb_n)(in_os)
in_h = Input(shape=[1], name = 'h')
emb_h = Embedding(max_h, emb_n)(in_h) 
in_d = Input(shape=[1], name = 'd')
emb_d = Embedding(max_d, emb_n)(in_d) 
in_wd = Input(shape=[1], name = 'wd')
emb_wd = Embedding(max_wd, emb_n)(in_wd) 
in_qty = Input(shape=[1], name = 'qty')
emb_qty = Embedding(max_qty, emb_n)(in_qty) 
in_c1 = Input(shape=[1], name = 'c1')
emb_c1 = Embedding(max_c1, emb_n)(in_c1) 
in_c2 = Input(shape=[1], name = 'c2')
emb_c2 = Embedding(max_c2, emb_n)(in_c2) 

In [30]:
type(in_app), type(emb_app)

(tensorflow.python.framework.ops.Tensor,
 tensorflow.python.framework.ops.Tensor)

In [44]:
in_app

<tf.Tensor 'app:0' shape=(?, 1) dtype=float32>

In [31]:
in_app.get_shape

<bound method Tensor.get_shape of <tf.Tensor 'app:0' shape=(?, 1) dtype=float32>>

In [32]:
emb_app.get_shape

<bound method Tensor.get_shape of <tf.Tensor 'embedding_1/Gather:0' shape=(?, 1, 50) dtype=float32>>

In [33]:
fe = concatenate([(emb_app), (emb_ch), (emb_dev), (emb_os), (emb_h), 
                 (emb_d), (emb_wd), (emb_qty), (emb_c1), (emb_c2)])

In [34]:
fe.get_shape

<bound method Tensor.get_shape of <tf.Tensor 'concatenate_1/concat:0' shape=(?, 1, 500) dtype=float32>>

In [35]:
s_dout = SpatialDropout1D(0.2)(fe)

In [36]:
s_dout.get_shape

<bound method Tensor.get_shape of <tf.Tensor 'spatial_dropout1d_1/cond/Merge:0' shape=(?, 1, 500) dtype=float32>>

In [37]:
x = Flatten()(s_dout)

In [38]:
x.get_shape

<bound method Tensor.get_shape of <tf.Tensor 'flatten_1/Reshape:0' shape=(?, ?) dtype=float32>>

In [39]:
x = Dropout(0.2)(Dense(dense_n,activation='relu')(x))
x = Dropout(0.2)(Dense(dense_n,activation='relu')(x))

In [40]:
outp = Dense(1,activation='sigmoid')(x)
model = Model(inputs=[in_app,in_ch,in_dev,in_os,in_h,in_d,in_wd,in_qty,in_c1,in_c2], outputs=outp)

In [41]:
batch_size = 20000
epochs = 2
exp_decay = lambda init, fin, steps: (init/fin)**(1/(steps-1)) - 1
steps = int(len(train_df) / batch_size) * epochs
lr_init, lr_fin = 0.001, 0.0001
lr_decay = exp_decay(lr_init, lr_fin, steps)
optimizer_adam = Adam(lr=0.001)#, decay=lr_decay)
model.compile(loss='binary_crossentropy',optimizer=optimizer_adam,metrics=['accuracy'])


In [42]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
app (InputLayer)                (None, 1)            0                                            
__________________________________________________________________________________________________
ch1 (InputLayer)                (None, 1)            0                                            
__________________________________________________________________________________________________
dev (InputLayer)                (None, 1)            0                                            
__________________________________________________________________________________________________
os (InputLayer)                 (None, 1)            0                                            
__________________________________________________________________________________________________
h (InputLa

In [73]:
from sklearn.metrics import roc_auc_score
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1, batch_size=100000):
        super(Callback, self).__init__()
        print('RocAuc evaluating batch size is: {}'.format(batch_size))
        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            print('\n on epoch end, start predicting validation set')
            y_pred = self.model.predict(self.X_val, batch_size=batch_size,verbose=1)
            print('\n start calculating ROC-AUC')
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch+1, score))

In [74]:
earlystopping = EarlyStopping(monitor="val_loss", mode="min", patience=3)

In [75]:
RocAuc = RocAucEvaluation(validation_data=(df_val, y_val), interval=1, batch_size=batch_size)

NameError: name 'df_val' is not defined

In [43]:
model.fit(train_df, y_train, batch_size=batch_size, epochs=2, shuffle=True, verbose=2)

ValueError: No data provided for "ch1". Need data for each key in: ['app', 'ch1', 'dev', 'os', 'h', 'd', 'wd', 'qty', 'c1', 'c2']

In [3]:
import pandas as pd

In [1]:
dtypes = {
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'hour'          : 'uint8',
        'is_attributed' : 'uint8', 
        'ip_day_hour_count': 'uint32', 
        'ip_os_day_hour_count': 'uint32', 
        'ip_app_day_hour_count': 'uint32', 
        'ip_app_os_day_hour_count': 'uint32', 
        'app_day_hour_count': 'uint32', 
        'ip_device_os_count': 'uint32', 
        'ip_app_device_os_count': 'uint32', 
        'ip_device_os_mean': 'float16',
        'ip_app_device_os_mean': 'float16',
        'ip_app_device_mean': 'float16',
        'app_device_os_mean': 'float16',
        'ip_device_os_time2nextclick': 'int32',
        'ip_app_device_os_time2nextclick': 'int32',
        'ip_app_device_time2nextclick': 'int32',
        'ip_device_os_time2previousclick': 'int32',
        'ip_app_device_os_time2previousclick': 'int32',
        'ip_app_device_time2previousclick': 'int32',
        'ip_device_os_countfromfuture': 'uint32', 
        'ip_app_device_os_countfromfuture': 'uint32', 
        'ip_app_device_countfromfuture': 'uint32', 
        'ip_device_os_countfrompast': 'uint32', 
        'ip_app_device_os_countfrompast': 'uint32', 
        'ip_app_device_countfrompast': 'uint32', 
        'ip_device_os_lasttimediff': 'int32',
        'ip_app_device_os_lasttimediff': 'int32',
        'ip_app_device_lasttimediff': 'int32',
        'ip_device_os_firsttimediff': 'int32',
        'ip_app_device_os_firsttimediff': 'int32',
        'ip_app_device_firsttimediff': 'int32',
        'matrixFact_user_iposdeviceapp_item_app': 'float16',
        'matrixFact_user_ip_item_appdeviceos': 'float16',
        'matrixFact_user_ipchannel_item_appdeviceos': 'float16',
        'ip_device_os_regression': 'float16',
        'ip_app_device_os_regression': 'float16',
        'ip_app_device_regression': 'float16',
        'ip_app_device_os_channel_regression': 'float16'
        } 

In [4]:
load_path = '/home/kai/data/kaggle/talkingdata/wl/data/equalhour/'
file_format = '{}_equalhour_supplementV1.csv'


#day7 = pd.read_csv(load_path+file_format.format('day7'),dtype=dtypes)
#print('load day7 done!')
day8 = pd.read_csv(load_path+file_format.format('day8'),dtype=dtypes)
print('load day8 done!')
day9 = pd.read_csv(load_path+file_format.format('day9'),dtype=dtypes)
print('load day9 done!')

load day8 done!
load day9 done!


In [5]:
day8.shape, day9.shape

((20446743, 49), (20898422, 49))

In [6]:
import sys

In [7]:
sys.getsizeof(day9)/1024**3

3.65907646715641

In [8]:
drop_cols = [col for col in day9.columns if 'countself' in col]
print(drop_cols)

['ip_day_hour_countself', 'ip_os_day_hour_countself', 'ip_app_day_hour_countself', 'ip_app_os_day_hour_countself', 'app_day_hour_countself', 'ip_device_os_countself', 'ip_app_device_os_countself']


In [9]:
day9.drop(drop_cols, axis=1, inplace=True)
day8.drop(drop_cols, axis=1, inplace=True)

In [10]:
day9.shape, day8.shape

((20898422, 42), (20446743, 42))

In [11]:
day9_s = day9.sample(frac=0.0001)

In [12]:
day9_s.shape

(2090, 42)

In [16]:
day9_s['ip_day_hour_count'].nunique()

777

In [18]:
for col in day9.columns:
    print(col, day9_s[col].nunique())

ip_day_hour_count 777
ip_os_day_hour_count 354
ip_app_day_hour_count 359
ip_app_os_day_hour_count 147
app_day_hour_count 182
ip_device_os_count 1324
ip_app_device_os_count 605
ip_device_os_mean 585
ip_app_device_os_mean 71
ip_app_device_mean 292
app_device_os_mean 443
ip_device_os_time2nextclick 440
ip_app_device_os_time2nextclick 951
ip_app_device_time2nextclick 837
ip_device_os_time2previousclick 471
ip_app_device_os_time2previousclick 1123
ip_app_device_time2previousclick 859
ip_device_os_countfromfuture 620
ip_app_device_os_countfromfuture 233
ip_app_device_countfromfuture 603
ip_device_os_countfrompast 825
ip_app_device_os_countfrompast 312
ip_app_device_countfrompast 781
ip_device_os_lasttimediff 1888
ip_app_device_os_lasttimediff 1551
ip_app_device_lasttimediff 1934
ip_device_os_firsttimediff 1994
ip_app_device_os_firsttimediff 1769
ip_app_device_firsttimediff 2000
matrixFact_user_iposdeviceapp_item_app 867
matrixFact_user_ip_item_appdeviceos 939
matrixFact_user_ipchannel_item_a

In [19]:
embedding_features = ['app','channel','device','os','hour']

In [20]:
train=day8.sample(frac=0.000001)
test=day9.sample(frac=0.000001)
train.shape, test.shape

((20, 42), (21, 42))

In [97]:
day9_s.head(2)

,ip_day_hour_count,ip_os_day_hour_count,ip_app_day_hour_count,ip_app_os_day_hour_count,app_day_hour_count,ip_device_os_count,ip_app_device_os_count,ip_device_os_mean,ip_app_device_os_mean,ip_app_device_mean,...,ip_device_os_regression,ip_app_device_os_regression,ip_app_device_regression,ip_app_device_os_channel_regression,app,channel,device,os,hour,is_attributed
11252805,20,10,6,5,407764,23,8,-0.010002,-0.010002,-0.010002,...,-0.500000,-0.5,-0.5,-0.5,12,328,1,25,10,0
4566799,166,55,30,14,478671,3680,358,0.002354,0.000000,0.000000,...,0.006641,0.0,0.0,0.0,2,477,1,19,5,0


In [105]:
day9_s.columns

Index(['ip_day_hour_count', 'ip_os_day_hour_count', 'ip_app_day_hour_count',
       'ip_app_os_day_hour_count', 'app_day_hour_count', 'ip_device_os_count',
       'ip_app_device_os_count', 'ip_device_os_mean', 'ip_app_device_os_mean',
       'ip_app_device_mean', 'app_device_os_mean',
       'ip_device_os_time2nextclick', 'ip_app_device_os_time2nextclick',
       'ip_app_device_time2nextclick', 'ip_device_os_time2previousclick',
       'ip_app_device_os_time2previousclick',
       'ip_app_device_time2previousclick', 'ip_device_os_countfromfuture',
       'ip_app_device_os_countfromfuture', 'ip_app_device_countfromfuture',
       'ip_device_os_countfrompast', 'ip_app_device_os_countfrompast',
       'ip_app_device_countfrompast', 'ip_device_os_lasttimediff',
       'ip_app_device_os_lasttimediff', 'ip_app_device_lasttimediff',
       'ip_device_os_firsttimediff', 'ip_app_device_os_firsttimediff',
       'ip_app_device_firsttimediff', 'matrixFact_user_iposdeviceapp_item_app',
       'mat

In [100]:
len_train = len(day8)
all_df = day8.append(day9)
del day8, day9; gc.collect()

3513

In [103]:
len(all_df), len_train

(41345165, 20446743)

In [104]:
print("label encoding....")
from sklearn.preprocessing import LabelEncoder
all_df[[ 'app', 'device', 'os', 'channel', 'hour']] = all_df[[ 'app', 'device', 'os', 'channel', 'hour']].apply(LabelEncoder().fit_transform)

label encoding....


embids = ['app', 'channel', 'device', 'os', 'hour', 'day', 'wday', 'qty', 'ip_app_count', 'ip_app_os_count']
# get the max of each code type
embmaxs = dict((col, np.max([train_df[col].max(), test_df[col].max()])+1) for col in embids)
def get_keras_data(dataset):
    X = dict((col, np.array(dataset[col])) for col in embids)
    return X
# Build the inputs, embeddings and concatenate them all for each column
emb_inputs = dict((col, Input(shape=[1], name = col))  for col in embids)
emb_model  = dict((col, Embedding(embmaxs[col], emb_n)(emb_inputs[col])) for col in embids)
fe = concatenate([(emb_) for emb_ in emb_model.values()])
# Rest of the model
....
model = Model(inputs=[inp for inp in emb_inputs.values()], outputs=outp)